In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
import requests
import csv
from datetime import datetime

In [ ]:
headers = {
    'accept' :'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'user_agent' : 'Mozlla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
    }
CSV = 'games.csv'
CSV_2 = 'info.csv'

In [ ]:
def get_content(params=''):
  host = 'https://steampricehistory.com'
  url = 'https://steampricehistory.com/popular?page='
  r = requests.get(url, headers=headers, params=params)
  soup = bs(r.text, 'html.parser')
  items = soup.find_all('tr')[1:]
  games = []
  titles = []

  for item in items:
    titles.append(item.find('a', class_='app-link').get_text(strip=True))
    games.append(
        {
            'title' : item.find('a', class_='app-link').get_text(strip=True),
            'recommended' : item.find_all('td')[4].get_text(strip=True),
            'current_price' : item.find_all('td')[2].get_text(strip=True),
            'discount' : item.find_all('td')[3].get_text(strip=True)
            }
            )
  return games, titles

Здесь с главной страницы собираются название игры, количество рекоммендаций, текущая цена и скидка. На выходе получаем список с играми и ссылки на каждую игру, чтоб потом их запарсить.

In [ ]:
#API_KEY и authorization - мои ключ и пароль для использования API
def full_game_info(game_title):
  url = 'https://api.igdb.com/v4/games'
  headers = {
      'Client-ID': API_KEY,
      'Authorization': f'Bearer {authorization}'
  }
  info = []
  try:
    try:
      try:
        data = f'search "{game_title}"; fields name, genres.name, involved_companies.company.name, aggregated_rating, alternative_names.name, collection.name, release_dates.date, franchises.name, game_engines.name, game_modes.name, platforms.name;'
        response = requests.post(url, headers=headers, data=data.encode('utf-8'))
        games = response.json()

        if len(games) > 0:
            game = games[0]  # Берем первую игру
            genre = game.get('genres', [{}])[0].get('name', 'N/A')
            publisher = game.get('involved_companies', [{}])[0].get('company').get('name')
            aggregated_rating = game.get('aggregated_rating', 'N/A')
            alternative_names = []
            for i in range(len(game.get('alternative_names', 'N/A'))):
              alternative_names.append(game.get('alternative_names', 'N/A')[i].get('name'))
            collection = game.get('collection', {}).get('name', 'N/A')
            release_date_unfixed = game.get('release_dates', [{}])[0].get('date', 'N/A')
            release_date = datetime.utcfromtimestamp(release_date_unfixed).strftime('%d-%m-%Y')
            game_engine = game.get('game_engines', [{}])[0].get('name', 'N/A')
            platforms = []
            for i in range(len(game.get('platforms', 'N/A'))):
              platforms.append(game.get('platforms', 'N/A')[i].get('name'))

            info.append(
                {
                    'Genre' : genre,
                    'Publisher' : publisher,
                    'Rating' : aggregated_rating,
                    'Names' : ', '.join(alternative_names),
                    'Collection' : collection,
                    'Release' : release_date,
                    'engine' : game_engine,
                    'Platforms' : ', '.join(platforms)
                }

            )

        else:
          pass
      except AttributeError:
        pass
    except TypeError:
      pass
  except KeyError:
    pass
  return info

In [ ]:
def csv_saver_common(items, path):
  with open(path, 'w', newline='') as file:
    writer = csv.writer(file, delimiter = ';')
    titles = ['Game title', 'Amount of recommendations', 'Current price', 'Current discount']
    writer.writerow(titles)
    for item in items:
      writer.writerow([item['title'], item['recommended'], item['current_price'], item['discount']])

In [ ]:
def csv_saver_info(data, path):
  with open(path, 'w', newline='') as file:
    writer = csv.writer(file, delimiter = ';')
    titles = ['Genre', 'Publisher', 'Rating', 'Alternative names', 'Collection', 'Release', 'Engine', 'Platforms']
    writer.writerow(titles)
    for i in data:
      writer.writerow([i['Genre'], i['Publisher'], i['Rating'], i['Names'], i['Collection'], i['Release'], i['engine'], i['Platforms']])

Данные записываются в csv

In [ ]:
def parser():
  pages = list(range(1, 123, 1))
  games = []
  game_info = []
  titles = []
  for page in pages:
    games.extend(get_content(params ={'page' : page})[0])
    titles.extend(get_content(params ={'page' : page})[1])
  csv_saver_common(games, CSV)
  return titles

In [ ]:
titles = parser()

Запускаются все функции и листаются страницы сайта

In [ ]:
game_info = []
for title in titles:
  game_info.append(full_game_info(str(title)))

In [ ]:
game_info

6100

In [ ]:
game_info_df = np.array(game_info)
info_df = pd.DataFrame()
cnt = 0
for lst in game_info_df:
  if len(lst) == 0:
    lst = [{
        'Genre' : '-',
        'Publisher' : '-',
        'Rating' : '-',
        'Names' : '-',
        'Collection' : '-',
        'Release' : '-',
        'engine' : '-',
        'Platforms' : '-'
        }]
    info_df = info_df.append(pd.DataFrame(lst))
  else:
    info_df = info_df.append(pd.DataFrame(lst))

In [ ]:
info_df.index = range(len(info_df))
info_df

,Genre,Publisher,Rating,Names,Collection,Release,engine,Platforms
0,Shooter,Hidden Path Entertainment,81.0,"Counter-Strike: Ofensiva Global, Counterstrike...",Counter-Strike,21-08-2012,Source,"Linux, PC (Microsoft Windows), PlayStation 3, ..."
1,Shooter,Bluehole Studio,75.0625,"Battlegrounds, Player Unknown, PUBG, PLAYERUNK...",N/A,20-12-2017,Unreal Engine 4,"PC (Microsoft Windows), PlayStation 4, Xbox On..."
2,-,-,-,-,-,-,-,-
3,-,-,-,-,-,-,-,-
4,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...
6095,Racing,Level 91 Entertainment,82.5,이너셜 드리프트,N/A,11-09-2020,N/A,"PC (Microsoft Windows), PlayStation 4, Xbox On..."
6096,-,-,-,-,-,-,-,-
6097,-,-,-,-,-,-,-,-
6098,-,-,-,-,-,-,-,-


In [ ]:
games_df = pd.read_csv('games.csv', sep=';')
total_df = pd.concat([games_df, info_df], axis = 1, ignore_index = False)
total_df

,Game title,Amount of recommendations,Current price,Current discount,Genre,Publisher,Rating,Names,Collection,Release,engine,Platforms
0,Counter-Strike: Global Offensive,"2,999,135",$0.00,0,Shooter,Hidden Path Entertainment,81.0,"Counter-Strike: Ofensiva Global, Counterstrike...",Counter-Strike,21-08-2012,Source,"Linux, PC (Microsoft Windows), PlayStation 3, ..."
1,PUBG: BATTLEGROUNDS,"1,382,744",$29.99,0,Shooter,Bluehole Studio,75.0625,"Battlegrounds, Player Unknown, PUBG, PLAYERUNK...",N/A,20-12-2017,Unreal Engine 4,"PC (Microsoft Windows), PlayStation 4, Xbox On..."
2,Rainbow Six Siege - Pro League Mute Set,"710,559",$19.99,0,-,-,-,-,-,-,-,-
3,Rainbow Six Siege - Pro League Bandit Set,"710,559",$19.99,0,-,-,-,-,-,-,-,-
4,Rainbow Six Siege - Pro League Thermite Set,"710,559",$19.99,0,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...
6095,Inertial Drift,301,$19.99,0,Racing,Level 91 Entertainment,82.5,이너셜 드리프트,N/A,11-09-2020,N/A,"PC (Microsoft Windows), PlayStation 4, Xbox On..."
6096,Survive on Raft,301,$9.99,0,-,-,-,-,-,-,-,-
6097,RainWallpaper,301,$3.99,0,-,-,-,-,-,-,-,-
6098,Seed of Evil,301,$6.49,-50%,-,-,-,-,-,-,-,-


In [ ]:
total_df = total_df.loc[total_df['Genre'] != '-']
total_df.index = range(len(total_df))
total_df

,Game title,Amount of recommendations,Current price,Current discount,Genre,Publisher,Rating,Names,Collection,Release,engine,Platforms
0,Counter-Strike: Global Offensive,"2,999,135",$0.00,0,Shooter,Hidden Path Entertainment,81.0,"Counter-Strike: Ofensiva Global, Counterstrike...",Counter-Strike,21-08-2012,Source,"Linux, PC (Microsoft Windows), PlayStation 3, ..."
1,PUBG: BATTLEGROUNDS,"1,382,744",$29.99,0,Shooter,Bluehole Studio,75.0625,"Battlegrounds, Player Unknown, PUBG, PLAYERUNK...",N/A,20-12-2017,Unreal Engine 4,"PC (Microsoft Windows), PlayStation 4, Xbox On..."
2,Tom Clancy's Rainbow Six Siege,"710,454",$19.99,0,Shooter,Ubisoft Montreal,74.166667,"R6:S, R6S, R6: Siege, 彩虹六号：围攻, Regenbogen 6 Be...",Rainbow Six,01-12-2020,AnvilNext,"PC (Microsoft Windows), PlayStation 4, Xbox On..."
3,Garry's Mod,"557,779",$9.99,0,Simulator,Facepunch Studios,N/A,"GMod, 게리 모드, gmod 9, gmod",N/A,24-12-2004,Source,"Linux, PC (Microsoft Windows), Mac"
4,Terraria,"536,708",$9.99,0,Platform,Headup Games,81.0,"Terraria: Journey's End, テラリア, 테라리아 (Terraria)...",Terraria,16-05-2011,N/A,"Linux, PC (Microsoft Windows), PlayStation 3, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2187,Yomawari: Midnight Shadows,303,$19.99,0,Puzzle,NIS America,83.0,深夜廻,N/A,24-10-2017,N/A,"PC (Microsoft Windows), PlayStation Vita, Play..."
2188,SiNKR,302,$3.99,0,Puzzle,Robert Wahler,68.333333,"シズめル, SiNKR",N/A,12-10-2017,N/A,"Linux, PC (Microsoft Windows), Mac, iOS, Xbox One"
2189,Invisigun Reloaded,302,$19.99,0,Arcade,Sombr Studio LLC,N/A,Invisigun Heroes,N/A,22-08-2019,N/A,"Linux, PC (Microsoft Windows), Mac, iOS, PlayS..."
2190,Tropico 3: Absolute Power,302,$9.99,0,Real Time Strategy (RTS),Haemimont Games,N/A,"Тропико 3: Абсолютная власть. Дополнение, トロピコ...",Tropico,17-05-2010,N/A,PC (Microsoft Windows)


In [ ]:
total_df.to_csv('total.csv', sep=';', index=False) #Получение итоговой таблицы